In [1]:
import pandas as pd
import numpy as np
import json
import urllib
import datetime
from time import sleep
import geopandas as gpd

In [2]:
def queryBicipublica(biciID):
    try:
        url = 'http://epok.buenosaires.gob.ar/getObjectContent/?id=estaciones_de_bicicletas%7C' + str(biciID)
        response = urllib.urlopen(url)
        rawData = response.read().decode("utf-8")
        parsedData = json.loads(rawData)

        bicisDisponibles = int(parsedData['contenido'][3]['valor'])
        posicionesDisponibles = int(parsedData['contenido'][4]['valor'])
        totales = int(parsedData['contenido'][5]['valor'])
    
    except:
        bicisDisponibles = np.nan
        posicionesDisponibles = np.nan
        totales = np.nan
        
    
    return {'bicisDisponibles':bicisDisponibles,'posicionesDisponibles':posicionesDisponibles,'totales':totales}

In [3]:
#base madre
geometria = gpd.read_file('../data/estaciones-de-bicicletas-rar/estaciones_de_bicicletas.shp')
barrios = gpd.read_file('../data/barrios-rar/barrios_badata.shp')
barrios = barrios.to_crs(epsg=4326)
geometria = geometria.to_crs(epsg=4326)

geometria = gpd.sjoin(geometria, barrios, how="left", op='intersects')




In [4]:
#poner un dia limite
diaLimite = 2

fechas = []
bicisDisponibles = []
posicionesDisponibles = []
totales = []
biciIDs = []



#inicia el loop para cargar data
#cambiar hora 10a 0 y hour por day
while True:
    if datetime.datetime.now().hour == 0:
        
        #mientras el dia actual sea menor al limite
        while datetime.datetime.now().day < 29:
            
            fecha = datetime.datetime.now()
            print 'fecha:',fecha
            #inicia el proceso para cada numero de estacion
            for biciID in geometria.NRO_EST:
                #agrego la id de la bici a la lista
                biciIDs.append(biciID)
                #creo el diccionario con datos
                diccio = queryBicipublica(biciID)
                #extraigo disponibles y total
                bicisDisponibles.append(diccio['bicisDisponibles'])
                posicionesDisponibles.append(diccio['posicionesDisponibles'])
                totales.append(diccio['totales'])
                
                #agrego fecha
                fechas.append(fecha)
                
            #espera 15 min
            print fecha
            sleep(60*15)
            #sleep(60)
        break
    else:
        sleep(60)

datos = pd.DataFrame({'NRO_EST':biciIDs,
                      'fecha':fechas,
                      'bicisDisponibles':bicisDisponibles,
                      'posicionesDisponibles':posicionesDisponibles,
                      'totales':totales
                      })

fecha: 2017-09-28 00:00:13.191112
2017-09-28 00:00:13.191112
fecha: 2017-09-28 00:17:46.461775
2017-09-28 00:17:46.461775
fecha: 2017-09-28 00:33:41.646607
2017-09-28 00:33:41.646607
fecha: 2017-09-28 00:49:32.857504
2017-09-28 00:49:32.857504
fecha: 2017-09-28 01:05:20.936895
2017-09-28 01:05:20.936895
fecha: 2017-09-28 01:21:10.128246
2017-09-28 01:21:10.128246
fecha: 2017-09-28 01:36:56.693935
2017-09-28 01:36:56.693935
fecha: 2017-09-28 01:52:47.901735
2017-09-28 01:52:47.901735
fecha: 2017-09-28 02:08:33.886171
2017-09-28 02:08:33.886171
fecha: 2017-09-28 02:24:19.030811
2017-09-28 02:24:19.030811
fecha: 2017-09-28 02:40:03.915993
2017-09-28 02:40:03.915993
fecha: 2017-09-28 02:55:50.518502
2017-09-28 02:55:50.518502
fecha: 2017-09-28 03:11:39.052845
2017-09-28 03:11:39.052845
fecha: 2017-09-28 03:27:23.675388
2017-09-28 03:27:23.675388
fecha: 2017-09-28 03:43:06.063667
2017-09-28 03:43:06.063667
fecha: 2017-09-28 03:58:52.832070
2017-09-28 03:58:52.832070
fecha: 2017-09-28 04:14:

In [5]:
geometria = geometria.merge(datos,how='left',on='NRO_EST')

In [6]:
#reemplazo
geometria.bicisDisponibles.fillna(-0.001,inplace=True)
geometria['disponibilidad'] = geometria.bicisDisponibles / geometria.totales *100
geometria['disponibilidad'][geometria['disponibilidad']>100] = 100.
geometria.disponibilidad.fillna(-0.001,inplace=True)


/home/pipe/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [7]:
geometria.fecha = geometria.fecha.map(str)
geometria.to_file('../data/28Septiembre')